#### **Deploy de aplicações**

#### **Observações**

#### **Conteúdo - Bases e Notebook da aula**

Github:  
https://github.com/FIAP/Pos_Tech_DTAT/tree/dd201a034223a16732c6f639b40600b26bd3129c

#### **Importação de bibliotecas**

#### **Funções (def)**

#### **Aula**